# Prepare Workspace

## Install required modules

In [1]:
#!pip install -U -q PyDrive
#!pip install geopandas

## Load required modules

In [2]:
import pandas as pd
import numpy as np

import datetime

import matplotlib.pyplot as plt
#import seaborn as sns
#from scipy import stats

#from google.colab import data_table
#data_table.enable_dataframe_formatter()
from sklearn.preprocessing import MinMaxScaler
#from sklearn.linear_model import LinearRegression
#from scipy.optimize import linprog
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


#from pydrive.auth import GoogleAuth
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#from google.colab import files

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [4]:
def VIF(X, threshold = 5):
  from statsmodels.stats.outliers_influence import variance_inflation_factor
  columns = X.columns.to_list()
  vif = pd.DataFrame()
  vif["VIF Factor"] = [variance_inflation_factor(X.values, i)\
                       for i in range(len(columns))]
  vif["features"] = X.columns

  ## Check for infinity VIF
  mask = (vif["VIF Factor"] == np.infty)
  if mask.any():
    to_drop=vif[mask]["features"].to_list()
    setColumns = set(columns)
    setTo_drop = set(to_drop)
    columns_new = list(setColumns.difference(setTo_drop))
    print(f"dropping due to infinite VIF:")
    print("\n".join(setTo_drop))
    return VIF(X[columns_new], threshold=threshold)

  # Drop feature with highest VIF if exceeding threshold
  if vif["VIF Factor"].max() > threshold:
    index = vif["VIF Factor"].idxmax()
    to_drop = vif.loc[index].to_dict()
    print("dropping {features} at index {index} with VIF of {VIF Factor:.2f} exceeding threshold of {threshold}"\
          .format(**to_drop, index=index, threshold=threshold))
    columns.remove(to_drop["features"])
    return VIF(X[columns], threshold)
  
  return vif

In [5]:
targets       = ['MAX_Completeness_pct', 'Weeks2MAX', 'IDX_W2M']
spatial       = ['Population Density (Per Sq. Mile)', 'Average Commute to Work (In Min)', 'AREA_SQMI', 'Density_Medical_Facilities_Population', 'Density_Medical_Facilities_Area', 'Density_Transportation_Stops_Pop', 'Density_Transportation_Stops_Area', 'Stops_per_Facility']
socioeconomic = ["% Total Population: Male","% Total Population: Female","% Total Population: Under 5 Years","% Total Population: 5 to 9 Years","% Total Population: 10 to 14 Years","% Total Population: 15 to 17 Years","% Total Population: 18 to 24 Years","% Total Population: 25 to 34 Years","% Total Population: 35 to 44 Years","% Total Population: 45 to 54 Years","% Total Population: 55 to 64 Years","% Total Population: 65 to 74 Years","% Total Population: 75 to 84 Years","% Total Population: 85 Years and Over","% Total Population: White Alone","% Total Population: Black or African American Alone","% Total Population: American Indian and Alaska Native Alone","% Total Population: Asian Alone","% Total Population: Two or More Races","% Population 25 Years and Over: Less than High School","% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","% Population 25 Years and Over: Some College or More","% Population 25 Years and Over: Bachelor's Degree or More","% Population 25 Years and Over: Master's Degree or More","% Population 25 Years and Over: Professional School Degree or More","% Population 25 Years and Over: Doctorate Degree","% Population 16 Years and Over: in Labor Force","% Population 16 Years and Over: in Labor Force: in Armed Forces","% Population 16 Years and Over: in Labor Force: Civilian","% Population 16 Years and Over: in Labor Force: Civilian: Employed","% Population 16 Years and Over: in Labor Force: Civilian: Unemployed","% Population 16 Years and Over: Not in Labor Force","% Households: Less than $25,000","% Households: $25,000 to $49,999","% Households: $50,000 to $74,999","% Households: $75,000 to $99,999","% Households: $100,000 or More","Median Household Income (In 2021 Inflation Adjusted Dollars)","Gini Index","% Workers 16 Years and Over: Car, Truck, or Van","% Workers 16 Years and Over: Drove Alone","% Workers 16 Years and Over: Carpooled","% Workers 16 Years and Over: Public Transportation (Includes Taxicab)","% Workers 16 Years and Over: Motorcycle","% Workers 16 Years and Over: Bicycle","% Workers 16 Years and Over: Walked","% Workers 16 Years and Over: Other Means","% Workers 16 Years and Over: Worked At Home","% Total: No Health Insurance Coverage","% Total: with Health Insurance Coverage","% Total: with Health Insurance Coverage: Public Health Coverage","% Total: with Health Insurance Coverage: Private Health Insurance","EP_POV150","EP_UNEMP","EP_HBURD","EP_NOHSDP","EP_UNINSUR","EP_AGE65","EP_AGE17","EP_SNGPNT","EP_LIMENG","EP_MINRTY","EP_MUNIT","EP_MOBILE","EP_CROWD","EP_NOVEH","EP_GROUPQ","F_POV150","F_UNEMP","F_HBURD","F_NOHSDP","F_UNINSUR","F_THEME1","F_AGE65","F_AGE17","F_DISABL","F_SNGPNT","F_LIMENG","F_THEME2","F_MINRTY","F_THEME3","F_MUNIT","F_MOBILE","F_CROWD","F_NOVEH","F_GROUPQ","F_THEME4","F_TOTAL",'%DAYPOP',"EP_NOINT","EP_AFAM","EP_HISP","EP_ASIAN","EP_AIAN","EP_TWOMORE","EP_OTHERRACE","%Total Child Population",'%Total University','%Total High school','%Total Less than High School','%Total Adult Population','%Total Elderly Population',"%Population with Disability","%LIMENG","%NOINT"]
others        = ["Total Population", 'Area (Land)',"Total Population:","Total Population:.1","EP_NHPI","Qualifying Name","File identification","State Postal Abbreviation","Summary Level","Geographic Component","Logical Record Number","US","Census Region","Census Division","State (Census Code)","State (FIPS Code)","County of current residence","County Subdivision (FIPS)","Place (FIPS Code)","Census Tract","Block Group","Consolidated City","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(Census)","American Indian Area/AlaskaNative Area/ Hawaiian Home Land(FIPS)","American Indian Trust Land/Hawaiian Home Land Indicator","American Indian TribalSubdivision (Census)","American Indian TribalSubdivision (FIPS)","% Total Population: Some Other Race Alone","Alaska Native RegionalCorporation (FIPS)","% Total Population: Native Hawaiian and Other Pacific Islander Alone","Metropolitan and MicropolitanStatistical Area","Combined Statistical Area","Metropolitan Statistical Area-Metropolitan Division","Metropolitan Area Central City","Metropolitan/MicropolitanIndicator Flag","New England City and Town Area","New England City and TownCombined Statistical Area","New England City and Town AreaDivision","Urban Area","Urban Area Central Place","Current Congressional District ***","State Legislative District Upper","State Legislative District Lower","Voting District","ZIP Code Tabulation Area (3-digit)","5-digit ZIP Code Tabulation Area","Subminor Civil Division (FIPS)","State-School District (Elementary)","State-School District (Secondary)","State-School District (Unified)","Urban/Rural","Principal City Indicator","Traffic Analysis Zone","Urban Growth Area","Public Use Microdata Area â€“ 5%File","Public Use Microdata Area - 1% File","Geographic Identifier","Area Name","Tribal Tract","Tribal Block Group","Place (State FIPS + Place FIPS)","Total Population: Male","Total Population: Female","Total Population: Under 5 Years","Total Population: 5 to 9 Years","Total Population: 10 to 14 Years","Total Population: 15 to 17 Years","Total Population: 18 to 24 Years","Total Population: 25 to 34 Years","Total Population: 35 to 44 Years","Total Population: 45 to 54 Years","Total Population: 55 to 64 Years","Total Population: 65 to 74 Years","Total Population: 75 to 84 Years","Total Population: 85 Years and Over","Total Population:.2","Total Population: White Alone","Total Population: Black or African American Alone","Total Population: American Indian and Alaska Native Alone","Total Population: Asian Alone","EP_DISABL","Total Population: Native Hawaiian and Other Pacific Islander Alone","Total Population: Some Other Race Alone","Total Population: Two or More Races","SPL_THEME1","RPL_THEME1","E_TOTPOP","Population 25 Years and Over:","SPL_THEME2","RPL_THEME2","SPL_THEME3","RPL_THEME3","SPL_THEME4","RPL_THEME4","SPL_THEMES","E_DAYPOP","RPL_THEMES","Population 25 Years and Over: Less than High School","Population 25 Years and Over: High School Graduate or More (Includes Equivalency)","Population 25 Years and Over: Some College or More","Population 25 Years and Over: Bachelor's Degree or More","Population 25 Years and Over: Master's Degree or More","Population 25 Years and Over: Professional School Degree or More","Population 25 Years and Over: Doctorate Degree","Population 16 Years and Over:","Population 16 Years and Over: in Labor Force","Population 16 Years and Over: in Labor Force: in Armed Forces","Population 16 Years and Over: in Labor Force: Civilian","Population 16 Years and Over: in Labor Force: Civilian: Employed","Population 16 Years and Over: in Labor Force: Civilian: Unemployed","Population 16 Years and Over: Not in Labor Force","Households:","Households: Less than $25,000","Households: $25,000 to $49,999","Households: $50,000 to $74,999","Households: $75,000 to $99,999","Households: $100,000 or More","Workers 16 Years and Over:","Workers 16 Years and Over: Car, Truck, or Van","Workers 16 Years and Over: Drove Alone","Workers 16 Years and Over: Carpooled","Workers 16 Years and Over: Public Transportation (Includes Taxicab)","Workers 16 Years and Over: Motorcycle","Workers 16 Years and Over: Bicycle","Workers 16 Years and Over: Walked","Workers 16 Years and Over: Other Means","Workers 16 Years and Over: Worked At Home","Total:","Total: No Health Insurance Coverage","Total: with Health Insurance Coverage","Total: with Health Insurance Coverage: Public Health Coverage","Total: with Health Insurance Coverage: Private Health Insurance","W","ST","STATE","ST_ABBR","STCNTY","COUNTY","LOCATION","M_TOTPOP","E_HU","M_HU","E_HH","M_HH","E_POV150","M_POV150","E_UNEMP","M_UNEMP","E_HBURD","M_HBURD","E_NOHSDP","M_NOHSDP","E_UNINSUR","M_UNINSUR","E_AGE65","M_AGE65","E_AGE17","M_AGE17","E_DISABL","M_DISABL","E_SNGPNT","M_SNGPNT","E_LIMENG","M_LIMENG","E_MINRTY","M_MINRTY","E_MUNIT","M_MUNIT","E_MOBILE","M_MOBILE","E_CROWD","M_CROWD","E_NOVEH","M_NOVEH","E_GROUPQ","M_GROUPQ","MP_POV150","MP_UNEMP","MP_HBURD","MP_NOHSDP","MP_UNINSUR","MP_AGE65","MP_AGE17","MP_DISABL","MP_SNGPNT","MP_LIMENG","MP_MINRTY","MP_MUNIT","MP_MOBILE","MP_CROWD","MP_NOVEH","MP_GROUPQ","E_NOINT","M_NOINT","E_AFAM","M_AFAM","E_HISP","M_HISP","E_ASIAN","M_ASIAN","E_AIAN","M_AIAN","E_NHPI","M_NHPI","E_TWOMORE","M_TWOMORE","E_OTHERRACE","M_OTHERRACE","MP_NOINT","MP_AFAM","MP_HISP","MP_ASIAN","MP_AIAN","MP_NHPI","MP_TWOMORE","MP_OTHERRACE","Count_Medical_Facilities","Count_Transportation_Stops","EPL_POV150","EPL_UNEMP","EPL_HBURD","EPL_NOHSDP","EPL_UNINSUR","EPL_AGE65","EPL_AGE17","EPL_DISABL","EPL_SNGPNT","EPL_LIMENG","EPL_MINRTY","EPL_MUNIT","EPL_MOBILE","EPL_CROWD","EPL_NOVEH","EPL_GROUPQ"]

## Read the file

In [6]:
df_scaled=pd.read_csv('df_counties.csv')
#files.download('df_scaled.csv')

In [7]:
df_scaled

,FIPS,MAX_Completeness_pct,Weeks2MAX,IDX_W2M,Population Density (Per Sq. Mile),Area (Land),Average Commute to Work (In Min),AREA_SQMI,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,...,%Total Child Population,%Total University,%Total High school,%Total Less than High School,%Total Adult Population,%Total Elderly Population,%Population with Disability,%LIMENG,%NOINT,%DAYPOP
0,56045,-0.009041,-1.465010,-1.455969,2.873639,2398.005019,27,2398.003892,0.001152,0.003336,...,0.202003,0.546811,0.373262,0.071137,0.592512,0.205485,0.134976,0.001873,0.150533,0.730913
1,56043,-0.009041,-1.465010,-1.455969,3.469903,2238.679370,12,2238.678265,0.000882,0.003127,...,0.227600,0.681173,0.256500,0.059008,0.549691,0.222709,0.133619,0.002017,0.129333,0.918946
2,56041,-0.009041,-1.465010,-1.455969,9.854347,2081.720793,21,2081.719808,0.000834,0.008166,...,0.286292,0.565556,0.364241,0.064309,0.566394,0.147314,0.150780,0.011632,0.070089,0.982870
3,56039,-0.009041,-1.465010,-1.455969,5.834346,3996.848665,16,3996.846846,0.001113,0.006505,...,0.177966,0.770429,0.168979,0.039696,0.667267,0.154766,0.063067,0.028601,0.099032,1.277274
4,56037,-0.009041,-1.465010,-1.455969,4.072032,10426.980103,20,10426.975770,0.000461,0.001918,...,0.257778,0.592896,0.326035,0.072944,0.615276,0.126946,0.130836,0.007405,0.097043,1.040252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,1009,-0.061571,-1.447915,-1.386344,91.308500,644.890660,34,644.890375,0.000554,0.049621,...,0.232372,0.480820,0.351234,0.163519,0.587885,0.179743,0.148282,0.013869,0.155606,0.730188
3094,1007,-0.061571,-1.447915,-1.386344,36.004970,622.469553,30,622.469288,0.000626,0.022491,...,0.205783,0.361961,0.439185,0.194619,0.633857,0.160361,0.167516,0.003352,0.171583,0.840216
3095,1005,-0.061571,-1.447915,-1.386344,28.540980,885.008060,26,885.007618,0.000559,0.015819,...,0.208401,0.383773,0.366769,0.243290,0.601528,0.190071,0.166187,0.017302,0.249700,1.084192
3096,1003,-0.055362,-1.593627,-1.538266,142.864400,1589.836160,26,1589.835294,0.000637,0.087430,...,0.215849,0.622558,0.273755,0.089858,0.578080,0.206071,0.140250,0.007298,0.112021,1.001457


In [8]:
df_scaled["Stops_per_Facility"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Medical_Facilities_Population"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Medical_Facilities_Area"].replace([-np.inf],np.nan, inplace=True)
df_scaled["Density_Transportation_Stops_Pop"].replace([np.inf],np.nan, inplace=True)
df_scaled["Density_Transportation_Stops_Area"].replace([np.inf],np.nan, inplace=True)

In [9]:
df_scaled[["Stops_per_Facility","Density_Medical_Facilities_Population","Density_Medical_Facilities_Area","Density_Transportation_Stops_Pop","Density_Transportation_Stops_Area"]]

,Stops_per_Facility,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,Density_Transportation_Stops_Pop,Density_Transportation_Stops_Area
0,0.0,0.001152,0.003336,0.0,0.0
1,0.0,0.000882,0.003127,0.0,0.0
2,0.0,0.000834,0.008166,0.0,0.0
3,0.0,0.001113,0.006505,0.0,0.0
4,0.0,0.000461,0.001918,0.0,0.0
...,...,...,...,...,...
3093,0.0,0.000554,0.049621,0.0,0.0
3094,0.0,0.000626,0.022491,0.0,0.0
3095,0.0,0.000559,0.015819,0.0,0.0
3096,0.0,0.000637,0.087430,0.0,0.0


In [10]:
df_scaled[np.isinf(df_scaled).any(axis=1)]

,FIPS,MAX_Completeness_pct,Weeks2MAX,IDX_W2M,Population Density (Per Sq. Mile),Area (Land),Average Commute to Work (In Min),AREA_SQMI,Density_Medical_Facilities_Population,Density_Medical_Facilities_Area,...,%Total Child Population,%Total University,%Total High school,%Total Less than High School,%Total Adult Population,%Total Elderly Population,%Population with Disability,%LIMENG,%NOINT,%DAYPOP
282,51081,-0.175545,-1.048116,-0.872572,38.83451,295.227067,23,295.22693,0.0,0.0,...,0.154993,0.358521,0.40742,0.228046,0.692979,0.152028,0.115934,0.003069,0.1675,1.027975


In [11]:
df_scaled.replace([np.inf], np.nan, inplace=True)

In [12]:
df_scaled.dropna(inplace=True)

## Scale

In [13]:
scaler = MinMaxScaler()
df_scaled1=df_scaled.copy()
#scaler.fit(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])
scaler.fit(df_scaled[spatial+socioeconomic])
df_scaled[spatial+socioeconomic] = scaler.transform(df_scaled[spatial+socioeconomic])
#df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])
#df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_scaled[['Average Commute to Work (In Min)', "%LIMENG"]])
df_scaled["% Total: No Health Insurance Coverage"] = df_scaled["% Total: No Health Insurance Coverage"].astype(float)/100 #move later to data preparation
df_scaled["%Total Child Population"] = df_scaled["%Total Child Population"]/100 #move later to data preparation

## Socio-economic Feature selection

In [109]:
X= df_scaled[socioeconomic+spatial]
y = df_scaled['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

                            OLS Regression Results                            
Dep. Variable:                IDX_W2M   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.115
Method:                 Least Squares   F-statistic:                     4.819
Date:                Fri, 14 Apr 2023   Prob (F-statistic):           1.14e-47
Time:                        16:22:11   Log-Likelihood:                -1748.5
No. Observations:                3043   AIC:                             3705.
Df Residuals:                    2939   BIC:                             4331.
Df Model:                         103                                         
Covariance Type:            nonrobust                                         
                                                                                          coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [15]:
X.isna().sum().max()

0

In [16]:
np.isinf(X).sum()

% Total Population: Male                 0
% Total Population: Female               0
% Total Population: Under 5 Years        0
% Total Population: 5 to 9 Years         0
% Total Population: 10 to 14 Years       0
                                        ..
Density_Medical_Facilities_Population    0
Density_Medical_Facilities_Area          0
Density_Transportation_Stops_Pop         0
Density_Transportation_Stops_Area        0
Stops_per_Facility                       0
Length: 113, dtype: int64

## Reduce columns with VIF

In [17]:
vif = VIF(df_scaled[socioeconomic], 10).round(2)
vif

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping due to infinite VIF:
F_NOHSDP
F_GROUPQ
% Total: with Health Insurance Coverage
F_DISABL
F_MUNIT
F_MINRTY
% Population 16 Years and Over: Not in Labor Force
F_TOTAL
% Population 25 Years and Over: Less than High School
%Total Elderly Population
F_NOVEH
F_UNEMP
F_THEME4
F_LIMENG
% Total Population: Female
% Total: No Health Insurance Coverage
F_HBURD
F_THEME2
F_THEME3
F_AGE17
F_AGE65
% Population 16 Years and Over: in Labor Force
F_CROWD
% Population 25 Years and Over: High School Graduate or More (Includes Equivalency)
F_UNINSUR
%Total Adult Population
% Total Population: Male
F_THEME1
F_SNGPNT
F_MOBILE
%Total Child Population
F_POV150
dropping % Workers 16 Years and Over: Car, Truck, or Van at index 64 with VIF of 315261050.28 exceeding threshold of 10
dropping % Workers 16 Years and Over: Drove Alone at index 63 with VIF of 85670127.74 exceeding threshold of 10
dropping % Population 25 Years and Over: Some College or More at index 32 with VIF of 74913368.35 exceeding threshol

,VIF Factor,features
0,3.46,EP_TWOMORE
1,5.28,EP_CROWD
2,1.91,EP_GROUPQ
3,4.79,EP_HISP
4,1.19,% Population 16 Years and Over: in Labor Force...
5,4.27,EP_MUNIT
6,1.74,EP_AIAN
7,1.60,EP_OTHERRACE
8,6.57,% Total Population: 85 Years and Over
9,4.82,%LIMENG


In [18]:
additional_features=['EP_DISABL', 
'%Population with Disability','EP_SNGPNT',
'% Population 25 Years and Over: Professional School Degree or More',
'%NOINT',
'% Workers 16 Years and Over: Carpooled']


In [19]:
reduced_features=list(vif['features'])
reduced_features

['EP_TWOMORE',
 'EP_CROWD',
 'EP_GROUPQ',
 'EP_HISP',
 '% Population 16 Years and Over: in Labor Force: in Armed Forces',
 'EP_MUNIT',
 'EP_AIAN',
 'EP_OTHERRACE',
 '% Total Population: 85 Years and Over',
 '%LIMENG',
 'EP_AFAM',
 '% Population 16 Years and Over: in Labor Force: Civilian: Unemployed',
 'EP_UNINSUR',
 '% Population 25 Years and Over: Doctorate Degree',
 '% Workers 16 Years and Over: Worked At Home',
 'EP_NOVEH',
 'EP_MOBILE',
 'EP_ASIAN',
 '% Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
 '% Workers 16 Years and Over: Bicycle',
 '% Workers 16 Years and Over: Other Means',
 '% Workers 16 Years and Over: Walked',
 '% Workers 16 Years and Over: Motorcycle']

### Feature engineering

In [20]:
df = df_scaled[targets].copy()

In [21]:
features=[]
for s in spatial:
  for p in (reduced_features+additional_features):
    df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
    features.append(s + "*" + p)

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[s + "*" + p]= df_scaled[s].copy()*df_scaled[p].copy()
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/950610936.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many t

In [22]:
df[reduced_features] = df_scaled[reduced_features]
df[additional_features] = df_scaled[additional_features]

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[reduced_features] = df_scaled[reduced_features]
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[reduced_features] = df_scaled[reduced_features]
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[reduced_features] = df_scaled[reduced_features]
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[additional_features] = df_scaled[additional_features]
/state/partition1/slurm_tmp/22363326.0.0/ipykernel_4786/4236574515.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many time

In [23]:
features += reduced_features
features += additional_features
features

['Population Density (Per Sq. Mile)*EP_TWOMORE',
 'Population Density (Per Sq. Mile)*EP_CROWD',
 'Population Density (Per Sq. Mile)*EP_GROUPQ',
 'Population Density (Per Sq. Mile)*EP_HISP',
 'Population Density (Per Sq. Mile)*% Population 16 Years and Over: in Labor Force: in Armed Forces',
 'Population Density (Per Sq. Mile)*EP_MUNIT',
 'Population Density (Per Sq. Mile)*EP_AIAN',
 'Population Density (Per Sq. Mile)*EP_OTHERRACE',
 'Population Density (Per Sq. Mile)*% Total Population: 85 Years and Over',
 'Population Density (Per Sq. Mile)*%LIMENG',
 'Population Density (Per Sq. Mile)*EP_AFAM',
 'Population Density (Per Sq. Mile)*% Population 16 Years and Over: in Labor Force: Civilian: Unemployed',
 'Population Density (Per Sq. Mile)*EP_UNINSUR',
 'Population Density (Per Sq. Mile)*% Population 25 Years and Over: Doctorate Degree',
 'Population Density (Per Sq. Mile)*% Workers 16 Years and Over: Worked At Home',
 'Population Density (Per Sq. Mile)*EP_NOVEH',
 'Population Density (Pe

In [24]:
len(features)

261

SyntaxError: invalid character in identifier (1805157811.py, line 18)

In [116]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

In [117]:
# Load the dataset
#data = load_boston()
X= df[features]
y = df['IDX_W2M']

In [118]:
# Define the Lasso regression model
lasso = Lasso()

In [119]:
# Use K-Fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [122]:
# Define the hyperparameter grid for tuning
param_grid = {
    'alpha': np.logspace(-5, 0, 20),
    'normalize': [True, False],
    'max_iter': [100, 500, 1000, 2000],
}

In [124]:
# Use GridSearchCV to find the best hyperparameters
lasso_cv = GridSearchCV(lasso, param_grid, cv=kf, scoring='neg_mean_squared_error', n_jobs=-1)
lasso_cv.fit(X, y)

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.00000000e-05, 1.83298071e-05, 3.35981829e-05, 6.15848211e-05,
       1.12883789e-04, 2.06913808e-04, 3.79269019e-04, 6.95192796e-04,
       1.27427499e-03, 2.33572147e-03, 4.28133240e-03, 7.84759970e-03,
       1.43844989e-02, 2.63665090e-02, 4.83293024e-02, 8.85866790e-02,
       1.62377674e-01, 2.97635144e-01, 5.45559478e-01, 1.00000000e+00]),
                         'max_iter': [100, 500, 1000, 2000],
                         'normalize': [True, False]},
             scoring='neg_mean_squared_error')

In [125]:
# Get the best hyperparameters
best_params = lasso_cv.best_params_
print("Best parameters found: ", best_params)

Best parameters found:  {'alpha': 0.00011288378916846884, 'max_iter': 500, 'normalize': True}


In [126]:
# Train the Lasso model with the best hyperparameters using cross-validation
best_lasso = Lasso(**best_params)
mse_scores = -cross_val_score(best_lasso, X, y, cv=kf, scoring='neg_mean_squared_error')
r2_scores = cross_val_score(best_lasso, X, y, cv=kf, scoring='r2')

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a S

In [127]:
print("Mean Squared Error (cross-validated): ", np.mean(mse_scores))
print("R2 Score (cross-validated): ", np.mean(r2_scores))

Mean Squared Error (cross-validated):  0.19591869730982509
R2 Score (cross-validated):  0.09230937517191903


/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/skle

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.169e+02, tolerance: 5.245e-02
  model = cd_fast.enet_coordinate_descent(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was depr

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.988e+02, tolerance: 5.260e-02
  model = cd_fast.enet_coordinate_descent(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was depr

...

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a S

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.846e+01, tolerance: 5.245e-02
  model = cd_fast.enet_coordinate_descent(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was depr

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of t

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.804e-01, tolerance: 5.269e-02
  model = cd_fast.enet_coordinate_descent(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was depr

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:148: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  warnings.warn(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.547e+01, tolerance: 5.260e-02
  model = cd_fast.enet_coordinate_descent(
/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was depr

## Regression without regularization

In [25]:
X= df[features]
y = df['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


                                 OLS Regression Results                                
Dep. Variable:                IDX_W2M   R-squared (uncentered):                   0.874
Model:                            OLS   Adj. R-squared (uncentered):              0.862
Method:                 Least Squares   F-statistic:                              73.76
Date:                Fri, 14 Apr 2023   Prob (F-statistic):                        0.00
Time:                        15:43:38   Log-Likelihood:                         -1658.5
No. Observations:                3043   AIC:                                      3839.
Df Residuals:                    2782   BIC:                                      5410.
Df Model:                         261                                                  
Covariance Type:            nonrobust                                                  
                                                                                                                  coef  

## Ridge Regression

In [107]:
# Prepare data
X = df[features]
y = df['IDX_W2M']

# Initialize K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define ridge regression model
ridge = Ridge()

# Specify alpha values for hyperparameter tuning
alphas = np.logspace(-5, 3, 500)
param_grid = {"alpha": alphas}

# Set up ridge regression with grid search for hyperparameter tuning
ridge_cv = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring="r2", cv=kf)
#ridge_cv = GridSearchCV(estimator=ridge, param_grid=param_grid, scoring="neg_root_mean_squared_error", cv=kf)

# Fit the ridge regression model to the data
ridge_cv.fit(X, y)

# Get the best alpha and corresponding mean squared error
best_alpha = ridge_cv.best_params_["alpha"]
best_mse = -ridge_cv.best_score_

print(f"Best alpha: {best_alpha}")
print(f"Best mean squared error: {best_mse}")


Best alpha: 0.3083031456568277
Best mean squared error: -0.08316723031009446


In [96]:
# Fit the ridge regression model with the best alpha to the entire dataset
ridge_best = Ridge(alpha=best_alpha)
ridge_best.fit(X, y)


Ridge(alpha=0.3083031456568277)

In [97]:
# Get the Ridge model's coefficients
ridge_coefficients = ridge_best.coef_


# Filter the features with non-zero coefficients (absolute value)
non_zero_ridge_coefficients = [(feature, coef) for feature, coef in zip(features, ridge_coefficients) if np.abs(coef) > 0.001]

# Create a DataFrame with the non-zero coefficients
non_zero_ridge_coefficients_df = pd.DataFrame(non_zero_ridge_coefficients, columns=["Feature", "Coefficient"])

In [98]:
pd.set_option('display.max_colwidth', None)
non_zero_ridge_coefficients_df

,Feature,Coefficient
0,Population Density (Per Sq. Mile)*EP_CROWD,-0.444178
1,Population Density (Per Sq. Mile)*EP_GROUPQ,0.036849
2,Population Density (Per Sq. Mile)*EP_HISP,0.473945
3,Population Density (Per Sq. Mile)*% Population 16 Years and Over: in Labor Force: in Armed Forces,-0.011732
4,Population Density (Per Sq. Mile)*EP_MUNIT,-0.436548
...,...,...
115,Density_Medical_Facilities_Area*EP_TWOMORE,0.252844
116,Density_Medical_Facilities_Area*EP_CROWD,0.112748
117,Density_Medical_Facilities_Area*EP_GROUPQ,0.406567
118,Density_Medical_Facilities_Area*EP_HISP,0.364892


In [99]:
ridge_features=list(non_zero_ridge_coefficients_df['Feature'])

In [106]:
X= df[ridge_features]
y = df['IDX_W2M']
#X = sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

                                 OLS Regression Results                                
Dep. Variable:                IDX_W2M   R-squared (uncentered):                   0.849
Model:                            OLS   Adj. R-squared (uncentered):              0.843
Method:                 Least Squares   F-statistic:                              137.3
Date:                Fri, 14 Apr 2023   Prob (F-statistic):                        0.00
Time:                        16:13:05   Log-Likelihood:                         -1927.5
No. Observations:                3043   AIC:                                      4095.
Df Residuals:                    2923   BIC:                                      4817.
Df Model:                         120                                                  
Covariance Type:            nonrobust                                                  
                                                                                                                  coef  

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [101]:
pred=slr.predict(X)
mse = mean_squared_error(df['IDX_W2M'], pred)
rmse = mean_squared_error(df['IDX_W2M'], pred, squared=False)
mape = mean_absolute_percentage_error(df['IDX_W2M'], pred)
table = pd.DataFrame({'MSE':[mse], 'RMSE':[rmse], 'MAPE':[mape]})
table

,MSE,RMSE,MAPE
0,0.182596,0.427313,0.797301


In [102]:
slr_as_html = slr_summary.tables[1].as_html()

stat=pd.read_html(slr_as_html, header=0, index_col=0)[0]

stat

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.8688,0.043,-20.093,0.000,-0.954,-0.784
Population Density (Per Sq. Mile)*EP_CROWD,-50.4498,124.297,-0.406,0.685,-294.168,193.268
Population Density (Per Sq. Mile)*EP_GROUPQ,9.8678,35.797,0.276,0.783,-60.322,80.058
Population Density (Per Sq. Mile)*EP_HISP,-20.4229,44.923,-0.455,0.649,-108.507,67.661
Population Density (Per Sq. Mile)*% Population 16 Years and Over: in Labor Force: in Armed Forces,255.1148,196.917,1.296,0.195,-130.995,641.224
...,...,...,...,...,...,...
Density_Medical_Facilities_Area*EP_TWOMORE,-18.7175,21.728,-0.861,0.389,-61.322,23.887
Density_Medical_Facilities_Area*EP_CROWD,190.1033,167.804,1.133,0.257,-138.922,519.128
Density_Medical_Facilities_Area*EP_GROUPQ,-7.6699,18.538,-0.414,0.679,-44.019,28.680
Density_Medical_Facilities_Area*EP_HISP,-17.5934,60.575,-0.290,0.772,-136.368,101.181


In [103]:
significant_features=stat.loc[stat['P>|t|'] <= 0.05]
significant_features

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.8688,0.043,-20.093,0.000,-0.954,-0.784
Population Density (Per Sq. Mile)*EP_AIAN,-964.2627,319.500,-3.018,0.003,-1590.731,-337.794
Population Density (Per Sq. Mile)*EP_UNINSUR,58.0131,19.352,2.998,0.003,20.068,95.958
Population Density (Per Sq. Mile)*% Workers 16 Years and Over: Worked At Home,47.0960,20.902,2.253,0.024,6.111,88.081
Population Density (Per Sq. Mile)*EP_NOVEH,-56.0621,27.910,-2.009,0.045,-110.787,-1.337
Population Density (Per Sq. Mile)*EP_MOBILE,-183.0220,41.026,-4.461,0.000,-263.465,-102.579
Population Density (Per Sq. Mile)*EP_ASIAN,-25.3459,10.741,-2.360,0.018,-46.406,-4.286
Population Density (Per Sq. Mile)*EP_DISABL,-12.4229,4.261,-2.916,0.004,-20.778,-4.068
Population Density (Per Sq. Mile)*%Population with Disability,440.3321,146.268,3.010,0.003,153.533,727.131
Average Commute to Work (In Min)*EP_MUNIT,-6.1402,1.462,-4.199,0.000,-9.007,-3.273


In [51]:
significant_features.reset_index(inplace=True)

In [52]:
active_features=list(significant_features['index'])

In [53]:
X= df[active_features]
y = df['IDX_W2M']
x= sm.add_constant(X)
slr = sm.OLS(y,X).fit()
slr_summary=slr.summary()
print(slr.summary())

                                 OLS Regression Results                                
Dep. Variable:                IDX_W2M   R-squared (uncentered):                   0.844
Model:                            OLS   Adj. R-squared (uncentered):              0.843
Method:                 Least Squares   F-statistic:                              681.8
Date:                Fri, 14 Apr 2023   Prob (F-statistic):                        0.00
Time:                        12:39:50   Log-Likelihood:                         -1977.9
No. Observations:                3043   AIC:                                      4004.
Df Residuals:                    3019   BIC:                                      4148.
Df Model:                          24                                                  
Covariance Type:            nonrobust                                                  
                                                                                                            coef    std 

/state/partition1/llgrid/pkg/anaconda/anaconda3-2022a/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)


In [54]:
slr_as_html = slr_summary.tables[1].as_html()

stat=pd.read_html(slr_as_html, header=0, index_col=0)[0]

stat

,coef,std err,t,P>|t|,[0.025,0.975]
Population Density (Per Sq. Mile)*% Total Population: 85 Years and Over,6.5595,2.119,3.095,0.002,2.404,10.715
Average Commute to Work (In Min)*% Population 16 Years and Over: in Labor Force: Civilian: Unemployed,5.4117,0.761,7.112,0.000,3.920,6.904
Average Commute to Work (In Min)*% Total Population: 85 Years and Over,-1.5356,0.300,-5.124,0.000,-2.123,-0.948
Average Commute to Work (In Min)*EP_AFAM,-1.2919,0.571,-2.263,0.024,-2.411,-0.173
Average Commute to Work (In Min)*EP_ASIAN,2.2316,0.471,4.740,0.000,1.308,3.155
Average Commute to Work (In Min)*EP_SNGPNT,-2.9798,0.288,-10.338,0.000,-3.545,-2.415
AREA_SQMI*% Population 16 Years and Over: in Labor Force: Civilian: Unemployed,8.4158,1.688,4.987,0.000,5.107,11.725
AREA_SQMI*EP_UNINSUR,-3.5845,0.900,-3.981,0.000,-5.350,-1.819
AREA_SQMI*EP_AFAM,-25.0505,3.462,-7.236,0.000,-31.839,-18.262
AREA_SQMI*EP_ASIAN,8.4534,2.447,3.455,0.001,3.655,13.251


In [39]:
stat.loc[stat['P>|t|'] <= 0.1]

,coef,std err,t,P>|t|,[0.025,0.975]
Population Density (Per Sq. Mile)*% Total Population: 85 Years and Over,7.6817,2.422,3.172,0.002,2.933,12.431
Average Commute to Work (In Min)*% Population 16 Years and Over: in Labor Force: Civilian: Unemployed,5.9997,0.824,7.280,0.000,4.384,7.616
Average Commute to Work (In Min)*% Total Population: 85 Years and Over,-1.4605,0.305,-4.794,0.000,-2.058,-0.863
Average Commute to Work (In Min)*EP_AIAN,-2.6388,1.583,-1.667,0.096,-5.742,0.465
Average Commute to Work (In Min)*EP_AFAM,-1.5507,0.583,-2.661,0.008,-2.693,-0.408
Average Commute to Work (In Min)*EP_ASIAN,2.1745,0.481,4.523,0.000,1.232,3.117
Average Commute to Work (In Min)*EP_SNGPNT,-2.9136,0.294,-9.919,0.000,-3.490,-2.338
AREA_SQMI*% Population 16 Years and Over: in Labor Force: Civilian: Unemployed,7.8711,1.719,4.578,0.000,4.500,11.242
AREA_SQMI*EP_UNINSUR,-3.4767,0.940,-3.697,0.000,-5.321,-1.633
AREA_SQMI*EP_AFAM,-25.0160,3.471,-7.208,0.000,-31.821,-18.211


# Predicting Tract

In [ ]:
df_tract=pd.read_csv('df_tracts.csv')

In [ ]:
df_tract[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_tract[['Average Commute to Work (In Min)', "%LIMENG"]])
#df_tract[['Average Commute to Work (In Min)', "%LIMENG"]] = scaler.transform(df_tract[['Average Commute to Work (In Min)', "%LIMENG"]])
df_tract["% Total: No Health Insurance Coverage"] = df_tract["% Total: No Health Insurance Coverage"].astype(float)/100 #move later to data preparation
df_tract["%Total Child Population"] = df_tract["%Total Child Population"]/100 #move later to data preparation

In [ ]:
df_tract.dropna(inplace=True)

In [ ]:
features

In [ ]:
df_pred=pd.DataFrame()
for s in spatial:
  for p in (reduced_features+additional_features):
    df_pred[s + "*" + p]= df_tract[s]*df_tract[p]

In [ ]:
predictions=slr.predict(df_pred[ridge_features])

In [ ]:
predictions_df = pd.DataFrame(predictions, columns=['IDX_W2M_Pred'])

In [ ]:
predictions_df["e**IDX"]=predictions_df["IDX_W2M_Pred"].apply(np.exp)

In [ ]:
result = df_tract.merge(predictions_df, how='left', left_index=True, right_index=True)

In [ ]:
result